## Data-driven Salary Analysis Using GlassDoor

Ricky Lim

In [1]:
import datetime

datetime.date.today().strftime("%B %d, %Y")

'December 22, 2017'

# Goal:

- What is the mean salary value for a researcher (onderzoeker) in The Netherlands ?
- What is the gap-ratio on the mean salary value between private and public sector on average in The Netherlands ?

# Methodology

- **Web-scraping** Glassdoor-specific query using `selenium` and `beautifulsoup (bs4)` python libraries
- **Extraction, Transformation, Loading (ETL)** salary data into DataFrame using `pandas` library
- **Visualization** using`bokeh`, and **descriptive statistics** on salary data using `pandas`

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

## Web-scraping

In [3]:
from get_gd_salary import get_all_salaries

WEBSITE = "https://www.glassdoor.com/Salaries/" + \
          "netherlands-onderzoeker-salary-SRCH_IL.0,11_IN178_KO12,23_SDMC.htm"
    
onderzoeker_salaris = get_all_salaries(WEBSITE)

## Extraction, Transformation, Loading (ETL)

In [4]:
import pandas as pd

salaris_pd = pd.DataFrame(onderzoeker_salaris, dtype="object")
salaris_pd.columns = ["job_title", "mean_salary"]
salaris_pd.head()
salaris_pd.describe()

,job_title,mean_salary
0,Onderzoeker - per maand bij Delft University o...,€ 2.126/mnd
1,Onderzoeker bij Delft University of Technology,€ 38.396
2,Onderzoeker - per maand bij Vrije Universiteit...,€ 2.473/mnd
3,Onderzoeker bij IMEC (Belgium),€ 49.814
4,Onderzoeker bij TNO,€ 46.246


,job_title,mean_salary
count,163,163
unique,163,162
top,Postdoctoraal Onderzoeker - maandcontract bij ...,€ 2.749/mnd
freq,1,2


In [5]:
# parse company data from job_title
salaris_pd["title"], salaris_pd["bedrijf"] = salaris_pd["job_title"].\
    str.split(" bij ", 1).str

In [6]:
salaris_pd.head()

,job_title,mean_salary,title,bedrijf
0,Onderzoeker - per maand bij Delft University o...,€ 2.126/mnd,Onderzoeker - per maand,Delft University of Technology
1,Onderzoeker bij Delft University of Technology,€ 38.396,Onderzoeker,Delft University of Technology
2,Onderzoeker - per maand bij Vrije Universiteit...,€ 2.473/mnd,Onderzoeker - per maand,Vrije Universiteit Amsterdam
3,Onderzoeker bij IMEC (Belgium),€ 49.814,Onderzoeker,IMEC (Belgium)
4,Onderzoeker bij TNO,€ 46.246,Onderzoeker,TNO


In [7]:
from re import sub

def annualize_salary(row):
    """
    unify salary values per year
    """
    # remove non-digit for dutch money (decimal point is "," instead of ".")
    money = sub(r"[^\d,]", "", row["mean_salary"])
    money = float(sub(r",", ".", money))
    title = row["title"]
    # convert monthly salary into annual with 12 times 
    if "mnd" in row["mean_salary"] or "maand" in title:
        money *= 12.0
        title = sub(r"- per maand", "", title)
    return title, money 

salaris_pd[["title", "mean_annual_salary"]] = salaris_pd.apply(annualize_salary, 
                                                               axis=1).apply(pd.Series)

# remove annual salary below minimum wages, assuming 1000 euros per maand
MIN_ANNUAL_SALARY = 12000
salaris_pd = salaris_pd[salaris_pd["mean_annual_salary"] >= MIN_ANNUAL_SALARY]

# aggregate salary values by company
company_pd = salaris_pd.groupby("bedrijf")["mean_annual_salary"].mean()
company_pd = pd.Series.to_frame(company_pd)
company_pd["company"] = company_pd.index
company_pd.head()

,mean_annual_salary,company
bedrijf,,
ABN AMRO,52427.0,ABN AMRO
AMC,50412.0,AMC
ASRE Onderzoek,82728.0,ASRE Onderzoek
Academisch Medisch Centrum,36632.0,Academisch Medisch Centrum
Advanced Medical Reviews,29784.0,Advanced Medical Reviews


In [8]:
# add label sector to a company, either public or private
import re

public = ["academisch", "academy", "university", "universiteit", 
          "hogeschool", "tu", "universitair", "umc", "ur", 
          "ministrie", "phd", "nwo", "rsm", "nations", 
          "gemeente", "rivm", "fontys", "fom", 
          "mc", "ministerie", "cwi",
          "society", "european", "agency", "netherlands"]

def ispublic(x):
    public_re = ""
    for _ in public:
        public_re += ".*%s.*|" % _
    public_re.rstrip("|")
    public_re = re.compile(public_re, re.IGNORECASE)
    if public_re.match(x).group():
        return "public" 
    return "private" 

company_pd["sector"] = company_pd.index.map(ispublic)
company_pd.head()

,mean_annual_salary,company,sector
bedrijf,,,
ABN AMRO,52427.0,ABN AMRO,private
AMC,50412.0,AMC,public
ASRE Onderzoek,82728.0,ASRE Onderzoek,private
Academisch Medisch Centrum,36632.0,Academisch Medisch Centrum,public
Advanced Medical Reviews,29784.0,Advanced Medical Reviews,private


## Visualization, and descriptive statistics

In [9]:
import math
from bokeh.models import ColumnDataSource

title="Mean Base Salary (Onderzoeker)" 
company_pd["color"] =["#abdda4" if s =="public" else 
                      "#2b83ba" for s in company_pd["sector"]]

source = ColumnDataSource(data=company_pd)

p = figure(x_range=list(company_pd["company"]), plot_width=1000, plot_height=800, 
           title=title)
p.vbar(x='company', top='mean_annual_salary', width=0.5, 
       source=source, legend="sector",
       line_color='white', fill_color="color")
p.xaxis.major_label_orientation = math.pi/2
p.yaxis.axis_label = "Mean Annual Salary in (€)"
p.xaxis.axis_label = "Company"
show(p)

company_pd.describe().round(2)

GlyphRenderer(id='59e01223-2f47-43c4-8d0f-cb8e97cb29f8', ...)

,mean_annual_salary
count,71.00
mean,42674.50
std,13793.15
min,19944.00
25%,31950.20
50%,39696.00
75%,51460.00
max,88153.00


# Results

The mean of base salary for a researcher position in the Netherlands across private and public sectors is **€ 42,671.03**

## Public and private sector-specific 

In [10]:
public_pd = company_pd[company_pd["sector"] == "public"]
private_pd = company_pd[company_pd["sector"] == "private"]

def salary_barplot(df, title):
    source = ColumnDataSource(data=df)
    p = figure(x_range=list(df["company"]), plot_width=1000, 
               plot_height=800, title=title)
    p.vbar(x='company', top='mean_annual_salary', width=0.8, 
           source=source, legend="sector",
           line_color='white', fill_color="color")
    p.xaxis.major_label_orientation = math.pi/2
    p.yaxis.axis_label = "Mean Annual Salary in (€)"
    p.xaxis.axis_label = "Company"
    show(p)

salary_barplot(public_pd, "Public Sector Mean Base Salary")
public_pd.describe().round(2)

,mean_annual_salary
count,44.00
mean,36813.64
std,9771.32
min,19944.00
25%,31045.75
50%,34390.33
75%,43894.38
max,60244.00


In [11]:
salary_barplot(private_pd, "Private Sector Mean Base Salary")
private_pd.describe().round(2)

,mean_annual_salary
count,27.00
mean,52225.53
std,14193.16
min,29784.00
25%,42710.50
50%,52427.00
75%,59982.50
max,88153.00


## Difference between private and public sector

In [12]:
import numpy as np
np.round(private_pd["mean_annual_salary"].mean()/
         public_pd["mean_annual_salary"].mean(), 2)

1.4199999999999999

The mean base salary value on average in *private* sector is **1.42x** higher than in *public* sector in the Netherlands, given this dataset

# Acknowledgement

- glassdoor: https://www.glassdoor.nl
- selenium: http://selenium-python.readthedocs.io/
- beautifulsoup: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
- pandas: https://pandas.pydata.org/